In [1]:
import sqlite3
import pandas as pd
from time import time

In [2]:
version2_path = "../data/raw/td_V2.db"
con2 = sqlite3.connect(version2_path)

In [88]:
def create_fts_table(cursor):
    cursor.execute("""CREATE VIRTUAL TABLE commit_texts USING fts3(
                    commit_hash VARCHAR(40) NOT NULL, 
                    message TEXT,
                    lines_added INT,
                    lines_removed INT,
                    files INT);""")
    
    cursor.execute("""INSERT INTO commit_texts
                    SELECT COMMIT_HASH as commit_hash, COMMIT_MESSAGE AS message, SUM(gcc.LINES_ADDED) AS lines_added, SUM(gcc.LINES_REMOVED) AS lines_removed, COUNT(DISTINCT gcc.FILE) as files
                    FROM GIT_COMMITS AS gc NATURAL JOIN GIT_COMMITS_CHANGES AS gcc
                    GROUP BY gc.COMMIT_HASH;""")

cursor = con2.cursor()
cursor.execute("DROP TABLE commit_texts")
create_fts_table(cursor)

In [105]:
def get_commits_from_issue(issue_id: str, connection: sqlite3.Connection) -> (pd.DataFrame):
    """This function returns a dataframe with the commits related to a single issue. This also extracts
    the number of lines changed.
        Input:
            - issue_key {str}: the issue key we use to match the descriptions
            - connection {sqlite3.Connection}: the connection to the database
        Output:
            - df {pandas.DataFrame}: a dataframe containing the following columns:
                 []
    """
    commits_query = f"SELECT * FROM commit_texts WHERE message MATCH '\"{issue_id}\"'"
    df = pd.read_sql(commits_query, connection)
    
    return (issue_id, df["lines_added"].sum(), df["lines_removed"].sum(), df["files"].sum())

In [108]:
get_commits_from_issue("MRM-127", con2)

('MRM-127', 5723, 5529, 152)

In [53]:
jira_query = """SELECT * FROM JIRA_ISSUES"""

df = pd.read_sql(jira_query, con2)

In [72]:
df.head()

,PROJECT_ID,KEY,PRIORITY,TYPE,SUMMARY,DESCRIPTION,CREATION_DATE,RESOLUTION_DATE,HASH,COMMIT_DATE,lines_added,lines_removed
0,org.apache:archiva,MRM-2000,Critical,Bug,Missing/invalid NOTICE file,There does not appear to be a NOTICE file alon...,2019-09-05 12:39:57 +0000,2019-10-04 22:06:00 +0000,76f47ea2033b09c145295799f6d8b64f7a02a77f,2019-10-03 17:37:37 +0200,0,0
1,org.apache:archiva,MRM-1987,Major,Project,Port security fixes for 2.2.4 to 3.0.0,Port the security fixes for CVE-2019-0213 and ...,2019-05-01 14:05:45 +0000,2019-05-03 19:43:41 +0000,8a13d818fe05aa13055fc1208da4764be109f872,2019-04-13 11:59:29 +0200,0,0
2,org.apache:archiva,MRM-1972,Minor,Bug,Stored XSS in Web UI Organization Name,UI Configuration->Configure appearance and the...,2018-02-22 23:38:28 +0000,2019-03-10 16:06:22 +0000,2ade46ab20f8fbc7d636cb520fc1af7094e633c8,2019-03-10 11:36:06 +0100,0,0
3,org.apache:archiva,MRM-1966,Minor,Bug,Yet another repository-purge not working,Hi guys!\r\n\r\nI'm having a bit of a struggle...,2017-11-24 13:23:12 +0000,2017-11-24 19:30:33 +0000,ae36964e7d354a0175352af1fb7ee70f1e4b5713,2017-11-24 20:29:17 +0100,0,0
4,org.apache:archiva,MRM-1956,Major,Task,Replace tomcat-jdbc pool with HikariCP,,2017-06-16 06:24:49 +0000,2017-06-20 11:49:55 +0000,8d3f82a479b6eb2bf709b1831351e20309735683,2017-06-20 21:49:28 +1000,0,0


In [93]:
a = df["KEY"].apply(get_commits_from_issue, args = (con2,))

In [94]:
df["lines_added"] = list(map(lambda x: x[1], a))
df["lines_removed"] = list(map(lambda x: x[2], a))
df["files_changed"] = list(map(lambda x: x[3], a))

In [97]:
df[(df["lines_added"] == 0) & (df["lines_removed"] == 0) & (df["files_changed"]==0)]

,PROJECT_ID,KEY,PRIORITY,TYPE,SUMMARY,DESCRIPTION,CREATION_DATE,RESOLUTION_DATE,HASH,COMMIT_DATE,lines_added,lines_removed,files_changed
0,org.apache:archiva,MRM-2000,Critical,Bug,Missing/invalid NOTICE file,There does not appear to be a NOTICE file alon...,2019-09-05 12:39:57 +0000,2019-10-04 22:06:00 +0000,76f47ea2033b09c145295799f6d8b64f7a02a77f,2019-10-03 17:37:37 +0200,0,0,0
1,org.apache:archiva,MRM-1987,Major,Project,Port security fixes for 2.2.4 to 3.0.0,Port the security fixes for CVE-2019-0213 and ...,2019-05-01 14:05:45 +0000,2019-05-03 19:43:41 +0000,8a13d818fe05aa13055fc1208da4764be109f872,2019-04-13 11:59:29 +0200,0,0,0
2,org.apache:archiva,MRM-1972,Minor,Bug,Stored XSS in Web UI Organization Name,UI Configuration->Configure appearance and the...,2018-02-22 23:38:28 +0000,2019-03-10 16:06:22 +0000,2ade46ab20f8fbc7d636cb520fc1af7094e633c8,2019-03-10 11:36:06 +0100,0,0,0
3,org.apache:archiva,MRM-1966,Minor,Bug,Yet another repository-purge not working,Hi guys!\r\n\r\nI'm having a bit of a struggle...,2017-11-24 13:23:12 +0000,2017-11-24 19:30:33 +0000,ae36964e7d354a0175352af1fb7ee70f1e4b5713,2017-11-24 20:29:17 +0100,0,0,0
4,org.apache:archiva,MRM-1956,Major,Task,Replace tomcat-jdbc pool with HikariCP,,2017-06-16 06:24:49 +0000,2017-06-20 11:49:55 +0000,8d3f82a479b6eb2bf709b1831351e20309735683,2017-06-20 21:49:28 +1000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29788,org.apache:httpcore,HTTPCORE-554,Minor,Improvement,Extension of the HTTP Status Code list with co...,The RFC 7538 describes following HTTP Status c...,2018-09-08 09:03:59 +0000,2018-09-08 17:01:20 +0000,a31a17a5f92c6c4cb0334ce46dbf10df8fb59bd5,2018-09-08 11:20:10 +0200,0,0,0
29789,org.apache:httpcore,HTTPCORE-553,Minor,Improvement,Extension of the HTTP Status Code list with co...,Add unsupported status codes from [https://too...,2018-09-08 08:46:25 +0000,2018-09-08 16:48:15 +0000,5a498afe027d2be76040dd26fc19e7b1fe447f1b,2018-09-08 11:27:08 +0200,0,0,0
29790,org.apache:httpcore,HTTPCORE-552,Minor,Improvement,Extension of the HTTP Status Code list with co...,The RFC 8297 describes following HTTP Status c...,2018-09-08 08:37:32 +0000,2018-09-08 16:35:57 +0000,18cbd6cdb6d553a7f1c6144f8f1478ef9e12fae3,2018-09-08 10:56:06 +0200,0,0,0
29791,org.apache:httpcore,HTTPCORE-551,Major,Task,Add server-side reactive bindings,This task is a continuation of HTTPCLIENT-1942...,2018-09-04 21:08:21 +0000,2018-09-09 13:56:31 +0000,98caba56e15a1c8fe28bb269c48f9092bcfd9e45,2018-09-01 12:58:12 -0700,0,0,0


In [104]:
pd.read_sql(f"SELECT * FROM GIT_COMMITS_CHANGES WHERE COMMIT_HASH = '18cbd6cdb6d553a7f1c6144f8f1478ef9e12fae3'", con2)


,PROJECT_ID,FILE,COMMIT_HASH,DATE,COMMITTER_ID,LINES_ADDED,LINES_REMOVED,NOTE
